# Advent of Code - 2024 - Day 15 - Problem 2

https://adventofcode.com/2024/day/15

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day15.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Map Class

In [2]:
CHAR_WALL = "#"
CHAR_BOX = "O"
CHAR_BOX_LEFT = "["
CHAR_BOX_RIGHT = "]"
CHAR_ROBOT = "@"
CHAR_EMPTY = "."


class Map:

    def __init__(self, lines):

        self._map_rows = list()
        for row in range(len(lines)):
            line = lines[row]
            map_row = list()
            self._map_rows.append(map_row)
            for col in range(len(line)):
                char = line[col]
                if char == CHAR_WALL:
                    map_row.extend([CHAR_WALL, CHAR_WALL])
                elif char == CHAR_BOX:
                    map_row.extend([CHAR_BOX_LEFT, CHAR_BOX_RIGHT])
                elif char == CHAR_ROBOT:
                    map_row.extend([CHAR_ROBOT, CHAR_EMPTY])
                else:
                    map_row.extend([CHAR_EMPTY, CHAR_EMPTY])

                if char == CHAR_ROBOT:
                    self._robot = (row, col * 2)

    def __str__(self):
        return "\n".join(["".join(line) for line in self._map_rows])

    def _get_value(self, row, col):
        return self._map_rows[row][col]

    def _set_value(self, row, col, value):
        self._map_rows[row][col] = value

    # Adds indexes to cols to ensure complete boxes have been selected.
    #
    def _add_missing_box_parts(self, row, cols):

        col = cols[0]
        value = self._get_value(row, col)
        if value == CHAR_BOX_RIGHT:
            cols.insert(0, col - 1)

        col = cols[-1]
        value = self._get_value(row, col)
        if value == CHAR_BOX_LEFT:
            cols.append(col + 1)

    # Determines if any of the specified locations contains a wall.
    #
    def _has_wall(self, row, cols):
        for col in cols:
            value = self._get_value(row, col)
            if value == CHAR_WALL:
                return True
        return False

    # Returns the list of columns containing boxes. Ensures that complete boxes are specified.
    #
    def _get_box_columns(self, row, cols):
        result = list()
        for col in cols:
            value = self._get_value(row, col)
            if value == CHAR_BOX_LEFT or value == CHAR_BOX_RIGHT:
                result.append(col)
        if result:
            self._add_missing_box_parts(row, result)
        return result

    # Moves all the specified blocks vertically.
    #
    def _move_boxes_vertically(self, row, cols, dy):

        next_row = row + dy

        # Fail if a wall is blocking any of the boxes we need to move.
        #
        if self._has_wall(next_row, cols):
            return False

        # Find and move the boxes we need to move out of the way first.
        #
        next_cols = self._get_box_columns(next_row, cols)
        if next_cols:
            if not self._move_boxes_vertically(next_row, next_cols, dy):
                return False

        # Swap values
        #
        for col in cols:
            temp = self._get_value(next_row, col)
            self._set_value(next_row, col, self._get_value(row, col))
            self._set_value(row, col, temp)

        return True

    def _move_boxes_horizontally(self, row, col, dx):
        next_col = col
        while True:
            next_col += dx
            value = self._get_value(row, next_col)
            if value == CHAR_WALL:
                return False
            elif value == CHAR_ROBOT:
                raise Exception("Ran into another robot!")
            elif value == CHAR_BOX_LEFT or value == CHAR_BOX_RIGHT:
                pass  # keep searching
            elif value == CHAR_EMPTY:
                #
                # Given:
                #
                #    [][]
                # 0123456789
                #
                # When col = 3, dx = 1, next_col = 7
                # When col = 6, dx = -1, next_col = 2
                #
                # range(7, 3, -1) / idx-1
                # range(2, 6, 1) / idx+1
                #
                for idx in range(next_col, col, -dx):
                    self._set_value(row, idx, self._get_value(row, idx - dx))
                self._set_value(row, col, CHAR_EMPTY)
                return True
            else:
                raise Exception(f"Invalid value {value} found.")

    def move_robot_vertically(self, dy):
        row = self._robot[0]
        col = self._robot[1]
        value = self._get_value(row + dy, col)

        if value == CHAR_WALL:
            return

        if value == CHAR_BOX_LEFT or value == CHAR_BOX_RIGHT:
            cols = [col]
            self._add_missing_box_parts(row + dy, cols)
            if not self._move_boxes_vertically(row + dy, cols, dy):
                return

        self._robot = (row + dy, col)
        self._set_value(row + dy, col, CHAR_ROBOT)
        self._set_value(row, col, CHAR_EMPTY)

    def move_robot_horizontally(self, dx):
        row = self._robot[0]
        col = self._robot[1]
        value = self._get_value(row, col + dx)

        if value == CHAR_WALL:
            return

        if value == CHAR_BOX_LEFT or value == CHAR_BOX_RIGHT:
            if not self._move_boxes_horizontally(row, col + dx, dx):
                return

        self._robot = (row, col + dx)
        self._set_value(row, col + dx, CHAR_ROBOT)
        self._set_value(row, col, CHAR_EMPTY)

    def get_gps_sum(self):
        result = 0
        for row in range(len(self._map_rows)):
            for col in range(len(self._map_rows[row])):
                if self._get_value(row, col) == CHAR_BOX_LEFT:
                    gps = (row * 100) + col
                    result += gps
        return result

## Parse Data

Parses the input `DATA` into `MAP` and `MOVES`.

In [3]:
idx = DATA.index("")

MAP = Map(DATA[:idx])
MOVES = "".join(DATA[idx + 1 :])

## Move Robot

In [4]:
for move in MOVES:
    if move == "<":
        MAP.move_robot_horizontally(-1)
    elif move == ">":
        MAP.move_robot_horizontally(1)
    elif move == "^":
        MAP.move_robot_vertically(-1)
    elif move == "v":
        MAP.move_robot_vertically(1)

print(MAP)
print("------")
print(f"GPC total = {MAP.get_gps_sum()}")

####################################################################################################
##..........[][][][]##......[]......####........####..........[]..........[][]....[]........[]....##
####..........[]....[][]..............[]..............[]..[]..[][][]..[]..[]......[][]..##....##[]##
####....##......[]..[]....##[]..[]..[][][]......##..................[]....................[]......##
##[]....##[]........[]..[][][]..[]........................[]........[]............##[]..[]........##
##[][]..[]##........[]......[]......[]....[]..[][]##....[]......[]......##......[]..........[][]..##
##[][].....[][].....[]....[]......##..[]..[]......[]........................[][]....[]..[]##[]..[]##
##..[]........##[][][]..[]##..........[]......[]............[]......##..........[]..[]..[]..##....##
##..........##[]..........[]...[].....[][]....[]....[]........##................##..[]............##
##......[]..[][]..........[][]..[]##[]..[][]....[]..##..##..[]........[]##..[][][]......[].